In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import itertools
import cv2
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [98]:
df = pd.read_csv('GroundTruth.csv')
df.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
df.shape

(10015, 8)

In [100]:
dfBCC = df[df["BCC"]==1]

In [101]:
dfBCC.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
25,ISIC_0024331,0.0,0.0,1.0,0.0,0.0,0.0,0.0
26,ISIC_0024332,0.0,0.0,1.0,0.0,0.0,0.0,0.0
39,ISIC_0024345,0.0,0.0,1.0,0.0,0.0,0.0,0.0
54,ISIC_0024360,0.0,0.0,1.0,0.0,0.0,0.0,0.0
97,ISIC_0024403,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [102]:
dfBCC.drop(columns=["MEL", "NV", "AKIEC", "BKL", "DF", "VASC"], inplace=True)

<ipython-input-102-7245fe19c3c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfBCC.drop(columns=["MEL", "NV", "AKIEC", "BKL", "DF", "VASC"], inplace=True)


In [103]:
dfBCC.head()

,image,BCC
25,ISIC_0024331,1.0
26,ISIC_0024332,1.0
39,ISIC_0024345,1.0
54,ISIC_0024360,1.0
97,ISIC_0024403,1.0


In [104]:
dfNotBCC = df[df["BCC"]==0]

In [105]:
dfNotBCC.tail()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10014,ISIC_0034320,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [106]:
dfNotBCC.drop(columns=["MEL", "NV", "AKIEC", "BKL", "DF", "VASC"], inplace=True)

<ipython-input-106-de857e9d030b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNotBCC.drop(columns=["MEL", "NV", "AKIEC", "BKL", "DF", "VASC"], inplace=True)


In [107]:
dfNotBCC.head()

,image,BCC
0,ISIC_0024306,0.0
1,ISIC_0024307,0.0
2,ISIC_0024308,0.0
3,ISIC_0024309,0.0
4,ISIC_0024310,0.0


In [108]:
dfNotBCC.shape

(9501, 2)

In [109]:
dfBCC.shape

(514, 2)

In [110]:
dfNotBCC = dfNotBCC.sample(frac = 0.06, random_state = 40)

In [111]:
dfBCC.shape

(514, 2)

In [112]:
dfNotBCC.shape

(570, 2)

In [116]:
df = pd.concat([dfBCC, dfNotBCC])

In [117]:
df.head()

,image,BCC
25,ISIC_0024331,1.0
26,ISIC_0024332,1.0
39,ISIC_0024345,1.0
54,ISIC_0024360,1.0
97,ISIC_0024403,1.0


In [118]:
df.shape

(1084, 2)

In [119]:
df = df.sample(frac=1, random_state=30)

In [120]:
df.head()

,image,BCC
9787,ISIC_0034093,1.0
9974,ISIC_0034280,0.0
1385,ISIC_0025691,1.0
2436,ISIC_0026742,0.0
3803,ISIC_0028109,0.0


In [121]:
imageArray = df["image"].to_numpy()

In [140]:
imageArray.shape

(1084,)

In [141]:
pwd

'/workspace/aman/MANYA'

In [142]:
cd images

/workspace/aman/MANYA/images


In [143]:
pwd

'/workspace/aman/MANYA/images'

In [144]:
Images = []

In [145]:
def processImage(image):
    image = cv2.resize(image, (150, 200))
    image = image.astype("float32") / 255
    return image

In [146]:
for img in imageArray:
    path = img + ".jpg"
    image = cv2.imread(path)
    image = processImage(image)
    Images.append(image)

In [147]:
len(Images)

1084

In [148]:
cd ..

/workspace/aman/MANYA


In [149]:
output = df["BCC"].to_numpy()

In [150]:
len(output)

1084

In [151]:
output.shape

(1084,)

In [152]:
xTrain, xTest, yTrain, yTest = train_test_split(np.array(Images), output, test_size = 0.2, random_state = 30)

In [153]:
xTrain.shape

(867, 200, 150, 3)

In [154]:
yTrain.shape

(867,)

In [155]:
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size = 0.2, random_state = 30)

In [156]:
xVal.shape

(174, 200, 150, 3)

In [157]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=(200, 150, 3)))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 150, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 150, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 75, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 100, 75, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 75, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 75, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 37, 64)        0

In [158]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["Accuracy"])

In [159]:
history = model.fit(xTrain, yTrain, validation_data = (xVal, yVal), epochs = 10, batch_size = 32)

Epoch 1/10
22/22 [==============================] - 6s 153ms/step - loss: 2.1614 - accuracy: 0.0000e+00 - val_loss: 0.6916 - val_accuracy: 0.0000e+00
Epoch 2/10
22/22 [==============================] - 1s 57ms/step - loss: 0.6954 - accuracy: 0.0000e+00 - val_loss: 0.6900 - val_accuracy: 0.0000e+00
Epoch 3/10
22/22 [==============================] - 1s 45ms/step - loss: 0.6796 - accuracy: 0.0000e+00 - val_loss: 0.6590 - val_accuracy: 0.0000e+00
Epoch 4/10
22/22 [==============================] - 1s 43ms/step - loss: 0.6291 - accuracy: 0.0000e+00 - val_loss: 0.6164 - val_accuracy: 0.0000e+00
Epoch 5/10
22/22 [==============================] - 1s 39ms/step - loss: 0.5619 - accuracy: 0.0000e+00 - val_loss: 0.5561 - val_accuracy: 0.0000e+00
Epoch 6/10
22/22 [==============================] - 1s 39ms/step - loss: 0.5380 - accuracy: 0.0000e+00 - val_loss: 0.6012 - val_accuracy: 0.0000e+00
Epoch 7/10
22/22 [==============================] - 1s 34ms/step - loss: 0.4894 - accuracy: 0.0000e+00 - 

In [160]:
predictions = model.predict(xTest)

In [161]:
preds = []

In [162]:
for p in predictions:
    if p>=0.5:
        preds.append(1)
    else:
        preds.append(0)

In [163]:
report = classification_report(yTest, preds, target_names = ["NOT NV", "NV"])

In [164]:
print(report)

              precision    recall  f1-score   support

      NOT NV       0.75      0.80      0.77       111
          NV       0.78      0.72      0.75       106

    accuracy                           0.76       217
   macro avg       0.76      0.76      0.76       217
weighted avg       0.76      0.76      0.76       217

